In [ ]:
from collections import namedtuple
import dataclasses as dc
import pathlib
import pprint
import random
import uuid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pendulum
import seaborn as sns

In [ ]:
import orchid  # For `unit_registry`
from orchid import (
    net_quantity as onq,
    project_store as ops,
    unit_system as units,
)

In [ ]:
import option
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet
# noinspection PyUnresolvedReferences
from System import DateTime

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), 
                                  project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: ops.ProjectStore(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
def get_project_wells(proj):
    return proj.Wells.Items

project_wells = toolz.pipe(
    projects,
    toolz.valmap(get_project_wells),
    toolz.valmap(lambda ws: {w.Name: w for w in ws}),
)
pprint.pprint(project_wells)

In [ ]:
toolz.pipe(project_wells.values(),
           toolz.map(len),
           list,)

In [ ]:
project_wells

In [ ]:
def get_well_stages(well):
    return well.Stages.Items

well_stages = toolz.pipe(
    project_wells,
    toolz.valmap(toolz.valmap(get_well_stages)),
    toolz.valmap(toolz.valmap(lambda ss: {s.DisplayStageNumber: s for s in ss})),
)
# pprint.pprint(well_stages)

In [ ]:
toolz.pipe(
    well_stages.values(),
    toolz.map(lambda wss: wss.values()),
    list,
    toolz.map(len),
    list,
)

In [ ]:
def get_stage_parts(stage):
    return stage.Parts

stage_parts = toolz.pipe(
    well_stages,
    toolz.valmap(toolz.valmap(toolz.valmap(get_stage_parts))),
)
# pprint.pprint(stage_parts)

In [ ]:
toolz.pipe(
    well_stages.values(),
    toolz.map(lambda wss: wss.values()),
    list,
    toolz.map(len),
    list,
)

In [ ]:
@dc.dataclass
class StagePart:
    object_id: uuid.UUID
    name: str
    display_name: str
    display_name_with_well: str
    display_name_without_well: str
    start_time: pendulum.DateTime
    stop_time: pendulum.DateTime
    isip: orchid.unit_registry.Quantity
    part_no: int
    
    
def isip_text(sp):
    isip = onq.as_measurement(
        units.as_unit_system(sp.Project.ProjectUnits).PRESSURE,
        option.maybe(sp.Isip),
    )
    result = f'{isip:~P}'
    return result

@toolz.curry
def summarize_stage_part(sp):
    result = StagePart(
        object_id=uuid.UUID(sp.ObjectId.ToString()),
        name=sp.Name,
        display_name=sp.DisplayName,
        display_name_with_well=sp.DisplayNameWithWell,
        display_name_without_well=sp.DisplayNameWithoutWell,
        start_time=sp.StartTime.ToString('o'),
        stop_time=sp.StopTime.ToString('o'),
        isip=isip_text(sp),
        part_no=sp.PartNumber)
    return result

In [ ]:
stage_part_summary = toolz.pipe(
    stage_parts,
    toolz.valmap(toolz.valmap(toolz.valmap(toolz.map(summarize_stage_part)))),
    toolz.valmap(toolz.valmap(toolz.valmap(list))),
    toolz.valmap(toolz.valmap(toolz.valmap(toolz.nth(0)))),  # Assume only single item in list
    toolz.valmap(toolz.valmap(toolz.valmap(lambda sp: dc.asdict(sp)))),
)
# pprint.pprint(stage_part_summary)

In [ ]:
def stage_part_summary_to_stage(sps):
    stage_no, summary = sps
    return toolz.merge({'stage_no': stage_no}, summary)

well_stages_summary = toolz.pipe(
    stage_part_summary,
    toolz.valmap(toolz.valmap(lambda ssps: ssps.items())),
    toolz.valmap(toolz.valmap(toolz.map(stage_part_summary_to_stage))),
    toolz.valmap(toolz.valmap(list)),
)
# pprint.pprint(well_stages_summary)

In [ ]:
def well_stages_summary_to_well(wss):
    well, summaries = wss
    result = toolz.pipe(
        summaries,
        toolz.map(lambda summary: toolz.merge({'well': well}, summary)),
        list,
    )
    return result

project_wells_summary = toolz.pipe(
    well_stages_summary,
    toolz.valmap(lambda wss: wss.items()),
    toolz.valmap(toolz.map(well_stages_summary_to_well)),
    toolz.valmap(toolz.concat),
    toolz.valmap(list),
)

# pprint.pprint(project_wells_summary)

In [ ]:
def project_wells_summary_to_project(pws):
    project, summaries = pws
    result = toolz.pipe(
        summaries,
        toolz.map(lambda summary: toolz.merge({'field': project}, summary)),
        list,
    )
    return result

projects_summary = toolz.pipe(
    project_wells_summary,
    lambda pwss: pwss.items(),
    toolz.map(project_wells_summary_to_project),
    toolz.concat,
    list,
)

# pprint.pprint(projects_summary)

In [ ]:
all_stage_parts_frame = pd.DataFrame(data=projects_summary)
# all_stage_parts_frame

In [ ]:
all_parts_identifiers = all_stage_parts_frame.loc[:, ['field', 'well', 'stage_no',
                                                      'part_no', 'name', 'display_name',
                                                      'display_name_with_well',
                                                      'display_name_without_well']]
# all_parts_identifiers

In [ ]:
# Fields
bakken = all_parts_identifiers['field'] == 'bakken'
montney = all_parts_identifiers['field'] == 'montney'

In [ ]:
# Wells
demo_1h = all_parts_identifiers['well'] == 'Demo_1H'
demo_2h = all_parts_identifiers['well'] == 'Demo_2H'
demo_4h = all_parts_identifiers['well'] == 'Demo_4H'
hori_01 = all_parts_identifiers['well'] == 'Hori_01'
hori_02 = all_parts_identifiers['well'] == 'Hori_02'
hori_03 = all_parts_identifiers['well'] == 'Hori_03'
vert_01 = all_parts_identifiers['well'] == 'Vert_01'

In [ ]:
def generate_candidates(count):
    return [n + 1 for n, _ in enumerate(range(count))]

candidates = {n: generate_candidates(n) for n in {50, 35, 15, 29, 28, 4}}
toolz.valmap(lambda cs: random.shuffle(cs), candidates)  # Relies on side-effect on `candidates`
sampled_stage_nos = toolz.valmap(lambda cs: cs[:8], candidates)
sampled_stage_nos

In [ ]:
def make_stage_selector(stage_no):
    return all_parts_identifiers['stage_no'] == stage_no

In [ ]:
# Bakken stages
stages_demo_1h = all_parts_identifiers['stage_no'].isin([1, 50, 31, 9])
stages_demo_2h = all_parts_identifiers['stage_no'].isin([1, 50, 42, 15])
stages_demo_4h = all_parts_identifiers['stage_no'].isin([1, 35, 18, 28])

# Montney stages
stages_hori_01 = all_parts_identifiers['stage_no'].isin([1, 15, 11, 4])
stages_hori_02 = all_parts_identifiers['stage_no'].isin([1, 29, 14, 21])
stages_hori_03 = all_parts_identifiers['stage_no'].isin([1, 28, 13, 19])
stages_vert_01 = all_parts_identifiers['stage_no'].isin([1, 2, 3, 4])

In [ ]:
pd.concat(
    objs=[
        all_parts_identifiers[bakken & demo_1h & stages_demo_1h],
        all_parts_identifiers[bakken & demo_2h & stages_demo_2h],
        all_parts_identifiers[bakken & demo_4h & stages_demo_4h],
    ], axis=0)

In [ ]:
pd.concat(
    objs=[
        all_parts_identifiers[montney & hori_01 & stages_hori_01],
        all_parts_identifiers[montney & hori_02 & stages_hori_02],
        all_parts_identifiers[montney & hori_03 & stages_hori_03],
        all_parts_identifiers[montney & vert_01 & stages_vert_01],
    ], axis=0)

In [ ]:
all_parts_details = all_stage_parts_frame.loc[:, ['field', 'well', 'stage_no', 'part_no',
                                                  'start_time', 'stop_time', 'isip']]
# all_parts_details

In [ ]:
pd.concat(
    objs=[
        all_parts_details[bakken & demo_1h & stages_demo_1h],
        all_parts_details[bakken & demo_2h & stages_demo_2h],
        all_parts_details[bakken & demo_4h & stages_demo_4h],
    ], axis=0)

In [ ]:
pd.concat(
    objs=[
        all_parts_details[montney & hori_01 & stages_hori_01],
        all_parts_details[montney & hori_02 & stages_hori_02],
        all_parts_details[montney & hori_03 & stages_hori_03],
        all_parts_details[montney & vert_01 & stages_vert_01],
    ], axis=0)